In [14]:
import joblib
import pandas as pd
import re, jieba

In [15]:
def processing(text):
    """
    数据预处理, 可以根据自己的需求进行重载
    """
    # 数据清洗部分
    text = re.sub("\{%.+?%\}", " ", text)  # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("@.+?( |$)", " ", text)  # 去除 @xxx (用户名)
    text = re.sub("【.+?】", " ", text)  # 去除 【xx】 (里面的内容通常都不是用户自己写的)
    text = re.sub("\u200b", " ", text)  # '\u200b'是这个数据集中的一个bad case, 不用特别在意
    # 分词
    words = [w for w in jieba.lcut(text) if w.isalpha()]
    # 对否定词`不`做特殊处理: 与其后面的词进行拼接
    while "不" in words:
        index = words.index("不")
        if index == len(words) - 1:
            break
        words[index: index + 2] = ["".join(words[index: index + 2])]  # 列表切片赋值的酷炫写法
    # 用空格拼接成字符串
    result = " ".join(words)
    return result


def load_corpus(aaa):
    """
    加载语料库
    """
    data = []
    for line in aaa:
        content = processing(line)
        data.append(content)
    return data

In [16]:
df = pd.read_csv('../csv/moredata_label.csv')
df.head()

,content,score
0,海南保亭的黎苗族风情景区，可以看到他们的艺术，居住环境，房屋，传统表演！大型艺术表演很有韵味...,0
1,槟榔谷，大家都知道槟榔，也知道海南人有吃槟榔的习惯，初到三亚，就听说出租车司机朝窗外吐槟榔，...,0
2,黎苗族文化主题公园，展示黎族、苗族服饰手工文化，传统民居，狩猎技能，传统舞蹈，民族美食，环境...,0
3,去的那天太阳很大，带着儿子一起去的，里面主要是少数民族的传统习俗文化遗产等，导游讲解一下会更...,0
4,该景点是海南游中比较值得去的地方，还有民族歌舞表演也值得看，这次和几家朋友自驾游，本来是想图...,0


In [17]:
df['content'] = load_corpus(df['content'])  # 进行数据清洗

## 划分测试集

In [18]:
df_train = df.sample(frac=0.8, replace=False, random_state=0, axis=0)
df_test = df[~df.index.isin(df_train.index)]
print(df_train, df_test)

                                                content  score
3684                                     主要 就是 参观 参观 而已      0
5453                          免费 的 海滩 环境 一般般 去 看看 椰林 也好      0
3140  三江 入海 的 玉带 滩 一边 的 江 平静 如 女孩 般 静谧 一边 的 大海 气势磅礴 ...      0
144   去过 呀 诺达 再 来 槟榔 谷 真的 觉得 此处 更 像是 大型 旅游 商店 游览车 每到...      0
2201                     不错 就是 交通 不太 方便 要是 有 专车 到 就 好 了      0
...                                                 ...    ...
6038                   假期 海滩 的 景色 不错 周末 的 时候 可以 来 这里 闲逛      0
5067  这里 应该 可以 算是 海口市 一处 较 安全 干净 的 海滩 里面 的 烤炉 较 多 是 ...      0
5586                             真的 不错 休闲 娱乐 为 一体 适合 游玩      0
6224                         海没 三亚 那么 蓝 毕竟 是 内海 不过 别有风味      0
5449                这里 推荐 过来 沙滩椅 元 三个 小时 有点像 阳光 一样 的 热情      1

[4995 rows x 2 columns]                                                 content  score
0     海南 保亭 的 黎 苗族 风情 景区 可以 看到 他们 的 艺术 居住 环境 房屋 传统 表...      0
7     槟榔 谷 顾名思义 这是 一个 以 槟榔 树为 主题 且 以 观赏 休闲 为 目的 的 一个...      0
21    这里 值得 一游 景色 很 美 节目 表演 完整

In [46]:
stopwords = []
with open("../stop.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
#vectorizer = CountVectorizer(max_df=100)
x_train = vectorizer.fit_transform(df_train['content'])
y_train = df_train['score']
x_test = vectorizer.transform(df_test['content'])
y_test = df_test['score']

C:\Users\Aris\anaconda3\envs\nlp\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['a', 'ain', 'aren', 'c', 'couldn', 'd', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'i', 'isn', 'lex', 'll', 'm', 'mon', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won', 'wouldn', 'β', 'δ', 'λ', 'ξ', 'ψ', 'в', 'ⅲ', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '元', '吨', '数', '日', '末', '０', '１', '１２', '２', '３', '５', 'ａ', 'ｂ', 'ｃ', 'ｅ', 'ｆ', 'ｌ', 'ｌｉ', 'ｒ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


In [60]:
print(vectorizer)

CountVectorizer(stop_words=['\ufeff$', '0', '1', '2', '3', '4', '5', '6', '7',
                            '8', '9', '?', '_', '“', '”', '、', '。', '《', '》',
                            '一', '一些', '一何', '一切', '一则', '一方面', '一旦', '一来',
                            '一样', '一般', '一转眼', ...],
                token_pattern='\\[?\\w+\\]?')


## 模型训练& 测试

In [48]:
# 训练
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pre = clf.predict(x_test)
print(y_pre)

# 测试
from sklearn import metrics

print(metrics.classification_report(y_test, y_pre))
print('准确率:', metrics.accuracy_score(y_test, y_pre))

[1 0 0 ... 0 0 0]
              precision    recall  f1-score   support

           0       0.85      0.97      0.91       964
           1       0.79      0.43      0.56       285

    accuracy                           0.84      1249
   macro avg       0.82      0.70      0.73      1249
weighted avg       0.84      0.84      0.83      1249

准确率: 0.8438751000800641


In [49]:
import joblib

joblib.dump(clf, 'clf_model.m')
print('ok!')

ok!


## 测试模型

In [54]:
t_clf = joblib.load('clf_model.m')  # 导入模型
print(t_clf)

MultinomialNB()


In [55]:
t_df = pd.read_csv('../csv/test_label.csv')

In [52]:
t_df_pre = t_clf.predict(vectorizer.transform(t_df['content']))

In [56]:
# t_df_pre =clf.predict(vectorizer.transform(t_df['content']))

In [59]:
print(t_df_pre,len(t_df_pre))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0] 307


In [57]:
from sklearn import metrics

print(metrics.classification_report(t_df['score'], t_df_pre))
print('准确率:', metrics.accuracy_score(t_df['score'], t_df_pre))

              precision    recall  f1-score   support

           0       0.45      0.96      0.61       140
           1       0.00      0.00      0.00       167

    accuracy                           0.44       307
   macro avg       0.22      0.48      0.31       307
weighted avg       0.20      0.44      0.28       307

准确率: 0.43973941368078173
